<a href="https://colab.research.google.com/github/HuyenNguyenHelen/MyTREC_2021/blob/main/BERT_Innitial_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers rank_bm25

     |████████████████████████████████| 92kB 5.5MB/s 
     |████████████████████████████████| 2.5MB 26.7MB/s 
     |████████████████████████████████| 1.2MB 43.6MB/s 
     |████████████████████████████████| 3.3MB 18.4MB/s 
     |████████████████████████████████| 901kB 58.0MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=a3f5967765212ecba47b5a37cc867e8425c52275a50c09b0b248102d101759aa
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


# Documents and Query processing

### Documents

In [3]:
import pandas as pd
path = r'/content/Bhanu_New_Summary_Combined_NER-BERN_Query2016_gene-disease-drug_2016_2000txt.csv'
with open(path, 'r', encoding = 'utf-8' ) as f:
    data = pd.read_csv(f, header = None, delimiter='\t')
data.columns = ['queryID','docID', 'score', 'rank', 'concat_str']
data = data.drop(columns =  ['queryID','score', 'rank' ] )
data1 = data.drop_duplicates()
set_doc = set(data1['docID'])
set_txt = set(data1['concat_str'])
print(len(data1['docID'].unique()))
print(len(data1['concat_str'].unique()))
print(len(set_doc), len(set_txt))

data1 = data1.drop_duplicates()
print(len(data1))
data1.head()


48122
48113
48122 48113
48122


,docID,concat_str
0,4529650,duoden adenocarcinoma melena duoden melena mic...
1,4798841,preoper cavern hemangioma present melena wirel...
2,3999026,anemia melena pyeloduoden fistula stoneassoci ...
3,3113951,ultrasonograph cow abomas lymphosarcoma descri...
4,3132483,cecal schwannoma gastrointestin bleed woman sc...


In [4]:

doc_txt = data1['concat_str'].values.tolist()

### Query

In [5]:
# Processed query 2016
# path = r"/content/Quries_2016_withTXT.txt"
# with open(path, 'r', encoding = 'utf-8' ) as f:
#     query_df = pd.read_csv(f, delimiter='\t')
# query_df.columns = ['QueryID', 'NERgdd']
# query_df['new_NERgdd'] = query_df['NERgdd'].apply(lambda x: ' '.join([i for i in str(x).split(',')]))

# query_df.head()


,QueryID,NERgdd,new_NERgdd
0,2,"right,hip,arthroplasti",right hip arthroplasti
1,3,"75f,hypoglycem",75f hypoglycem
2,4,osteoporosi,osteoporosi
3,5,"cough,product",cough product
4,6,"atrial,pe,fibril",atrial pe fibril


In [12]:
# Loading the Query [2016/2021]
## Query 2016
import pandas as pd
import xml.etree.ElementTree as ET

tree = ET.parse(r"/content/topics2016.xml")
root = tree.getroot()

queryID, note, description, summary=[],[],[], []

 
i=0
while i<len(root):
    queryID.append(i+1)
    for j in range (len(root[i])):
        if root[i][j].tag == 'note':
            note.append(root[i][j].text.replace('\n',''))
        if root[i][j].tag == 'summary':
            summary.append(root[i][j].text)
        if root[i][j].tag == 'description':
            description.append(root[i][j].text)
  
    i+=1
    
query_df = pd.DataFrame(zip(queryID, note, description, summary), columns =['queryID', 'note', 'description', 'summary'] )
query_df.head()

,queryID,note,description,summary
0,1,78 M w/ pmh of CABG in early [**Month (only...,78 M transferred to nursing home for rehab aft...,A 78 year old male presents with frequent stoo...
1,2,Ms [**Known patient lastname 241**] is a [*...,An elderly female with past medical history of...,An elderly female with past medical history of...
2,3,Pt is a 75F with a PMHx significant for sev...,"A 75F with a PMHx significant for severe PVD, ...",A 75F found to be hypoglycemic with hypotensio...
3,4,The patient is an 87 yo woman with h/o oste...,"An 87 yo woman with h/o osteoporosis, multiple...","An 87 yo woman with h/o osteoporosis, DM2, dem..."
4,5,"An 82 M with COPD, s/p bioprosthetic AVR fo...",An 82 man with chronic obstructive pulmonary d...,An 82 man with multiple chronic conditions and...


In [13]:
# Processing query
queries = query_df['summary'].values.tolist()

# Question-Answering using Simple Wikipedia Index

This examples demonstrates the setup for Query / Question-Answer-Retrieval.

You can input a query or a question. The script then uses semantic search
to find relevant passages in Simple English Wikipedia (as it is smaller and fits better in RAM).

For semantic search, we use SentenceTransformer('msmarco-distilbert-base-v2') and retrieve
100 potentially passages that answer the input query.

Next, we use a more powerful CrossEncoder (cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6')) that
scores the query and all retrieved passages for their relevancy. The cross-encoder is neccessary to filter out certain noise
that might be retrieved from the semantic search step.


In [7]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

# We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
model_name = 'msmarco-MiniLM-L-6-v3'
bi_encoder = SentenceTransformer(model_name)
top_k = 1000  # Number of passages we want to retrieve with the bi-encoder

# The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
        
# Here, we compute the corpus_embeddings from scratch (which can take a while depending on the GPU)
corpus_embeddings = bi_encoder.encode(doc_txt, convert_to_tensor=True, show_progress_bar=True)
print(corpus_embeddings)


tensor([[ 0.0024, -0.2993,  0.0318,  ..., -0.3888, -0.2327,  0.1130],
        [ 0.0950,  0.0926,  0.4330,  ..., -0.4781,  0.3871,  0.3569],
        [ 0.2041, -0.1071, -0.0122,  ..., -0.1918,  0.2932,  0.3275],
        ...,
        [ 0.1110,  0.5613,  0.3377,  ..., -0.2529,  0.0881,  0.2810],
        [ 0.1753, -0.3144,  0.3600,  ...,  0.0781, -0.1704,  0.3538],
        [-0.2366, -0.1217,  0.2195,  ..., -0.0954,  0.1245,  0.5587]],
       device='cuda:0')


In [ ]:
txt = 'avail urea nitrogencreatinin gastrointestin bleed melena aim urea nitrogen creatinin buncr distinguish upper gastrointestin bleed gib differenti upper gastrointestin bleed ugib present melena retrospect gib present melena review demograph modal transfus ugib gastrointestin bleed lgib 583 233 ugib varic 371 peptic ulcer pud 314 buncr ugib lgib odd 69 confid interv 13372 ugib buncr varic pud 0015 buncr appear unit varic pud 10313 pud meckel diverticulum buncr uneven differenti ugib lgib melena buncr interpret care gastrointestinalbleedingmelenachild introductionmateri methodsresultsclin enrol patientsclin ugib lgib melena patientsdiagnost modal subjectslaboratori melena ugib lgiblaboratori melena causesr transfus gib melenadiscuss melena refer tarri black stool gastrointestin pediatr melena upper gastrointestin bleed ugib ligament tretiz intestin proxim colon bleed melena slow brisk bleed 50100 ml bleed stomach continu bleed stop fast massiv bleed hematochezia advanc bleed tarri black stool highrat bleed esophag varic duoden ulcer dieulafoy arterioven malform meckel diverticulum melena esophag varic peptic ulcer pud meckel diverticulum bleed hemodynam stabil priorit bleed proper initi bleed hematemesi melena abdomin locat gastrointestin gastrointestin lavag present lavag esophag bleed give discomfort yield duoden bleed urea nitrogen creatinin buncr differenti ugib gastrointestin bleed lgib urea nitrogen gastrointestin bleed absorpt intestin excret urea nitrogen hypovolem buncr impli ugib pud bleed melena bleed bun insignific aforement research gib differenti diver bleed avail buncr distinguish ugib lgib caus ugib melena buncr distinguish ugib lgib help local bleed gib present melena melena buncr buncr differenti ugib lgib gib present melena buncr 69 ugib lgib buncr manifest ugib varic pud pud ugib meckel diverticulum lgib buncr local bleed gib felber buncr ugib ugib lgib gib ugib buncr 98 688 buncr ratio ugib lgib lgib buncr 486 ugib 143 lgib 125 gib buncr 36 impli ugib bleed hardli local pediatr age birth 180 buncr rang 108 124 median 320 ugib 104 354 median 225 lgib use buncr buncr differenti ugib lgib buncr pud rang 135 381 median varic rang 176 124 median 400 ugib melena buncr differ ugib buncr pud attribut zero buncr melena gib pud buncr meckel diverticulum buncr pud meckel diverticulum cleveland age birth old ugib underw egd prolaps gastropathi 127 erosionsulc 108 eros esophag duoden erosionsulc 82 esophag varic 63 ugib gastritiserosionulc 358 duodenitisduoden ulcer 76 esophag varic 72 hassoon ugib esophag varic 39 erosionulc 293 duoden ulcer 74 esophag 49 ugib esophagogastr varic 351 gastritisgastr ulcer 162 duodenitisduoden ulcer 216 esophag 54 varic bleed hassoon lgib account 226 melena melena meckel diverticulum gib melena differ hospit gib 1115 lowest younger old ugib induc melena 297 270 216 month 135 81 610 month lgib 429 610 meckel diverticulum rafeey esophag varic bleed 04 younger 67 esophag varic rare younger old varic bleed age 43 youngest varic bleed portal vein agenesi varic bleed lymphocyt count pud meckel diverticulum influenc preced fever cough 538 varic bleed gib melena custom pud antacid except duoden ulcer bleed endoscop endoscop varic ligat histoacryl inject varic bleed hemoclip treat dieulafoy lesion snare polypectomi colon polyp meckel diverticulum duoden ulcer arterioven malform bowel arterioven malform gastrointestin stromal huge duoden polyp gib melena stop spontan sometim intens repetit continu lifethreaten consid hemodynam reserv younger ass restor hemodynam transfus improv deterior perfus enhanc suppli lifesav restrict transfus ugib gdl hemoglobin liber transfus gdl sharma gib refus bloodderiv ethic religi concern transfusionfre safe refus transfus nevertheless 18yearold appli hemodynam reserv transfus done 517 501 ugib varic bleed 889 lgib meckel diverticulum melena melena underw transfus melena statist insignific 182 pud underw transfus varic 692 meckel diverticulum 667 transfus 571 hemoglobin gdl belong restrict retrospect undiagnos possibl bia buncr classifi buncr buncr prove distinguish bleed ugib lgib gib melena buncr appear differ uneven ugib varic pud pud insignificantli lgib meckel diverticulum buncr appear varic pud pud mickel diverticulum impli ugib confound lgib melena depend transit gastrointestin tract duoden bleed observ interpret appli buncr distinguish ugib lgib largescal one'
test = data[data['concat_str']==txt]['docID'].values
type(test.astype(str))
test

array([], dtype=int64)

In [16]:
##### Sematic Search
all_output = []
for id, query in zip(query_df['queryID'], queries):
    
  # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits_en = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    for hit in hits_en:
      tempt_dic = []
      output = {}
      #hit = sorted(hits, key=lambda x: x['score'], reverse=True)
      for h in hit:   
        #temp_dict = {'doc': [], 'score': []}
        tempt_dic.append([data[data['concat_str']==doc_txt[h['corpus_id']]]['docID'].values[0], h['score']] )
        with open('/content/semantic_output_f2k_rawq.txt', "a") as outFile:
            outFile.write(str(id) + "\t" + str(data[data['concat_str']==doc_txt[h['corpus_id']]]['docID'].values[0]) + '\t' + str(h['score']) + "\n")
      output[id] = tempt_dic
      
      
    all_output.append(output)


all_output[0]

{1: [[2515291, 0.4102191925048828],
  [4068079, 0.34852710366249084],
  [2135206, 0.3384253978729248],
  [3762039, 0.3272421658039093],
  [4210724, 0.30595436692237854],
  [4285825, 0.3046540915966034],
  [4205677, 0.2986425459384918],
  [4125334, 0.29487985372543335],
  [3840666, 0.2941548228263855],
  [3900824, 0.29373908042907715],
  [4136954, 0.2914067804813385],
  [4391998, 0.29067739844322205],
  [4591626, 0.2898426353931427],
  [2135104, 0.28978025913238525],
  [4334886, 0.2856503129005432],
  [3290313, 0.2842830419540405],
  [4717365, 0.28266438841819763],
  [3663238, 0.28256386518478394],
  [4094716, 0.28226104378700256],
  [4553188, 0.280398428440094],
  [4707593, 0.2794201970100403],
  [4612480, 0.2790078818798065],
  [3714416, 0.2784312069416046],
  [2871548, 0.2780696153640747],
  [4519295, 0.2778099775314331],
  [3290021, 0.2762410044670105],
  [2646742, 0.27547502517700195],
  [3028987, 0.2753024697303772],
  [2397000, 0.27359697222709656],
  [4451808, 0.2735036313533783

In [9]:
#### Cross-encoding retrieval

for id, query in zip(query_df['QueryID'], queries):

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits_cross = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    for hit in hits_cross:
      print('--------------11111111----------------', hit)
      ##### Re-Ranking #####
      # Now, score all retrieved passages with the cross_encoder
      #print(hit)
      #print([doc_txt[h['corpus_id']] for h in hit])
      cross_inp = [[query, doc_txt[h['corpus_id']]] for h in hit]
      cross_scores = cross_encoder.predict(cross_inp)

      # Sort results by the cross-encoder scores
      for idx in range(len(cross_scores)):
          hit[idx]['cross-score'] = cross_scores[idx]
      print('------222222222222-------', hit)
      tempt_dic = []
      output = {}
      #hit = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
      for h in hit:   
        #temp_dict = {'doc': [], 'score': []}
        tempt_dic.append([data[data['concat_str']==doc_txt[h['corpus_id']]]['docID'].values[0], h['cross-score']] )
        with open('/content/rerank_cross_output_f2k_rawq.txt', "a") as outFile:
            outFile.write(str(id) + "\t" + str(data[data['concat_str']==doc_txt[h['corpus_id']]]['docID'].values[0]) + '\t' + str(h['cross-score']) + "\n")
      output[id] = tempt_dic



#     hits_re = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
#     for hit in hits_re[0:3]:
#         print("\t{:.3f}\t{}".format(hit['cross-score'], doc_txt[hit['corpus_id']].replace("\n", " ")))
#     #return hits_re['cross-score'], doc_txt[int(hits_re['corpus_id'])]
      

--------------11111111---------------- [{'corpus_id': 624, 'score': 0.5958926677703857}, {'corpus_id': 559, 'score': 0.5712393522262573}, {'corpus_id': 1697, 'score': 0.5511196851730347}, {'corpus_id': 608, 'score': 0.5501634478569031}, {'corpus_id': 571, 'score': 0.5193310976028442}, {'corpus_id': 1645, 'score': 0.505456805229187}, {'corpus_id': 528, 'score': 0.497448593378067}, {'corpus_id': 524, 'score': 0.495950847864151}, {'corpus_id': 549, 'score': 0.4945206046104431}, {'corpus_id': 573, 'score': 0.4937623143196106}, {'corpus_id': 931, 'score': 0.4928174614906311}, {'corpus_id': 544, 'score': 0.4897884130477905}, {'corpus_id': 555, 'score': 0.4890596270561218}, {'corpus_id': 741, 'score': 0.48434293270111084}, {'corpus_id': 981, 'score': 0.481203556060791}, {'corpus_id': 629, 'score': 0.47906258702278137}, {'corpus_id': 1068, 'score': 0.474350243806839}, {'corpus_id': 1210, 'score': 0.4708772301673889}, {'corpus_id': 570, 'score': 0.4689294099807739}, {'corpus_id': 859, 'score': 

In [ ]:
# This function will search all wikipedia articles for passages that
# answer the query
# def search(query):

#     ##### Sematic Search #####
#     # Encode the query using the bi-encoder and find potentially relevant passages
#     question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
#     question_embedding = question_embedding.cuda()
#     hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
#     hits = hits[0]  # Get the hits for the first query

#     ##### Re-Ranking #####
#     # Now, score all retrieved passages with the cross_encoder
#     cross_inp = [[query, doc_txt[hit['corpus_id']]] for hit in hits]
#     cross_scores = cross_encoder.predict(cross_inp)

#     # Sort results by the cross-encoder scores
#     for idx in range(len(cross_scores)):
#         hits[idx]['cross-score'] = cross_scores[idx]

#     # Output of top-5 hits from bi-encoder
#     print("\n-------------------------\n")
#     print("Top-3 Bi-Encoder Retrieval hits")
#     hits_en = sorted(hits, key=lambda x: x['score'], reverse=True)
#     for hit in hits_en[0:3]:
#          print("\t{:.3f}\t{}".format(hit['score'], doc_txt[hit['corpus_id']].replace("\n", " ")))

#     # Output of top-5 hits from re-ranker
#     print("\n-------------------------\n")
#     print("Top-3 Cross-Encoder Re-ranker hits")
#     hits_re = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
#     for hit in hits_re[0:3]:
#         print("\t{:.3f}\t{}".format(hit['cross-score'], doc_txt[hit['corpus_id']].replace("\n", " ")))
#     #return hits_re['cross-score'], doc_txt[int(hits_re['corpus_id'])]




